<a href="https://colab.research.google.com/github/LeThanhLong2001/SentimentAnalysis-NaiveBayes/blob/main/SentimentAnalysis_NaiveBayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Naive Bayes Classfier for Sentiment Analysis on Tweets
**Phân tích cảm xúc trên tập 1Tweets1 sử dụng thuật toán phân loại Naive Bayes**

In [1]:
import re
import string
import nltk
import numpy as np
nltk.download('twitter_samples')
from nltk.corpus import twitter_samples
from nltk.tokenize import TweetTokenizer
from tqdm import tqdm

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.


In [24]:
# Tải về tập dữ liệu tweets
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

# Chia thành 2 tập train và test
# train: 4000 samples, test: 1000 samples
train_pos = all_positive_tweets[:4000]
test_pos = all_positive_tweets[4000:]

train_neg = all_negative_tweets[:4000]
test_neg = all_negative_tweets[4000:]

train_x = train_pos + train_neg
test_x = test_pos + test_neg

# Tao nhan negative:0, positive:1
train_y = np.append(np.ones(len(train_pos)), np.zeros(len(train_neg)))
test_y = np.append(np.ones(len(test_pos)), np.zeros(len(test_neg)))

# Data Preprocessing

In [3]:
all_positive_tweets[:10]

['#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)',
 '@Lamb2ja Hey James! How odd :/ Please call our Contact Centre on 02392441234 and we will be able to assist you :) Many thanks!',
 '@DespiteOfficial we had a listen last night :) As You Bleed is an amazing track. When are you in Scotland?!',
 '@97sides CONGRATS :)',
 'yeaaaah yippppy!!!  my accnt verified rqst has succeed got a blue tick mark on my fb profile :) in 15 days',
 '@BhaktisBanter @PallaviRuhail This one is irresistible :)\n#FlipkartFashionFriday http://t.co/EbZ0L2VENM',
 "We don't like to keep our lovely customers waiting for long! We hope you enjoy! Happy Friday! - LWWF :) https://t.co/smyYriipxI",
 '@Impatientraider On second thought, there’s just not enough time for a DD :) But new shorts entering system. Sheep must be buying.',
 'Jgh , but we have to go to Bayan :D bye',
 'As an act of mischievousness, am calling the ETL layer of our in-house warehousing 

In [5]:
def basic_preprocess(text):
    text = re.sub(r'\$\w*', '', text)

    text = re.sub(r'^RT[\s]+', '', text)

    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text)

    text = re.sub(r'#', '', text)
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    text_tokens = tokenizer.tokenize(text)

    text_clean = []
    for word in text_tokens:
        if word not in string.punctuation:
            text_clean.append(word)

    return text_clean

In [6]:
example_sentence = "RT @Twitter @chapagain Hello There! Have a great day. #good #morning http://chapagain.com.np"
basic_preprocess(example_sentence)

['hello', 'there', 'have', 'a', 'great', 'day', 'good', 'morning']

#Buid Vocab

In [10]:
def count_freq_words(corpus, labels):
    """ Xây dựng bộ từ điển tần suất xuất hiện của các từ
    Args:
        corpus: tập danh sách các câu
        labels: tập nhãn tương ứng với các câu trong corpus (0 hoặc 1)
    Output:
        model: bộ từ điển ánh xạ mỗi từ và tần suất xuất hiện của từ đó trong corpus
            key: (word, label)
            value: frequency
            VD: {('boring', 0): 2} => từ boring xuất hiện 2 lần trong các sample thuộc class 0
    """
    model = {}
    for label, sentence in zip(labels, corpus):
        for word in basic_preprocess(sentence):
            # Định nghĩa key của model là tuple (word, label)
            pair = (word, label)
            # Nếu key đã tồn tại trong model thì tăng value lên 1
            if pair in model:
                model[pair] += 1
            # Nếu key chưa tồn tại trong model thì bổ sung key vào model với value =1
            else:
                model[pair] = 1
    return model

In [15]:
freqs = count_freq_words(train_x, train_y)
freqs

{('followfriday', 1.0): 23,
 ('for', 1.0): 606,
 ('being', 1.0): 49,
 ('top', 1.0): 29,
 ('engaged', 1.0): 7,
 ('members', 1.0): 11,
 ('in', 1.0): 381,
 ('my', 1.0): 441,
 ('community', 1.0): 25,
 ('this', 1.0): 242,
 ('week', 1.0): 61,
 (':)', 1.0): 2847,
 ('hey', 1.0): 60,
 ('james', 1.0): 7,
 ('how', 1.0): 60,
 ('odd', 1.0): 1,
 (':/', 1.0): 5,
 ('please', 1.0): 77,
 ('call', 1.0): 21,
 ('our', 1.0): 111,
 ('contact', 1.0): 4,
 ('centre', 1.0): 1,
 ('on', 1.0): 242,
 ('02392441234', 1.0): 1,
 ('and', 1.0): 553,
 ('we', 1.0): 182,
 ('will', 1.0): 150,
 ('be', 1.0): 198,
 ('able', 1.0): 6,
 ('to', 1.0): 836,
 ('assist', 1.0): 1,
 ('you', 1.0): 1187,
 ('many', 1.0): 28,
 ('thanks', 1.0): 311,
 ('had', 1.0): 35,
 ('a', 1.0): 725,
 ('listen', 1.0): 8,
 ('last', 1.0): 36,
 ('night', 1.0): 50,
 ('as', 1.0): 82,
 ('bleed', 1.0): 2,
 ('is', 1.0): 354,
 ('an', 1.0): 99,
 ('amazing', 1.0): 39,
 ('track', 1.0): 5,
 ('when', 1.0): 69,
 ('are', 1.0): 152,
 ('scotland', 1.0): 2,
 ('congrats', 1.0)

In [16]:
# Hàm lấy ra tần suất xuất hiện là value trong `freq` dựa vào key (word, label)
def lookup(freqs, word, label):
    '''
    Args:
        freqs: a dictionary with the frequency of each pair
        word: the word to look up
        label: the label corresponding to the word
    Output:
        count: the number of times the word with its corresponding label appears.
    '''
    count = 0

    pair = (word, label)
    if pair in freqs:
        count = freqs[pair]

    return count

lookup(freqs, "just", 0)

197

# Naive Bayes Algorithm

In [17]:
def compute_prior_prob(train_y):
    # Tính D, D_pos, D_neg dựa vào x_train
    ### START CODE HERE
    # Tính D, số lượng các sample trong training
    D = len(train_y)

    # Tính D_pos, số lượng các positive sample trong training
    D_pos = len(list(filter(lambda x: x == 1, train_y)))

    # Tính D_neg, số lượng các negative sample trong training
    D_neg = len(list(filter(lambda x: x == 0, train_y)))

    ### END CODE HERE
    # Tính xác suất tiên nghiệm cho các class 0 và 1
    p_prior = {0:(D_neg/D), 1:(D_pos/D)}
    return p_prior

In [18]:
compute_prior_prob(train_y)

{0: 0.5, 1: 0.5}

In [19]:
def compute_likelihood(freqs):
    # Tính xác suất likelihood của mỗi từ trong bộ từ điển

    ### START CODE HERE
    # Tính V các từ duy nhất xuất hiện trong tập train
    vocab = set([pair[0] for pair in freqs.keys()])
    V = len(vocab)

    # Tính N_pos: số lượng từ trong positive samples và N_neg: số từ trong negative sample
    N_pos = N_neg = 0
    for pair in freqs.keys():
        # Nếu như class: 1 tăng N_pos thêm số lần xuất hiện của pair trong freqs
        if pair[1] > 0:
            N_pos += freqs[pair]

        # Nếu như class: 0 tăng N_neg thêm số lần xuất hiện của pair trong freqs
        else:
            N_neg += freqs[pair]

    print(f'V: {V}, N_pos: {N_pos}, N_neg: {N_neg}')

    # Tính likelihood cho mỗi từ trong bộ từ điển
    p_likelihood = {}
    for word in vocab:
        # Lấy tần xuất xuất hiện của mỗi từ là positive hoặc negative
        freq_pos = lookup(freqs, word, 1)
        freq_neg = lookup(freqs, word, 0)

        # Tính xác suất likelihood của mỗi từ với class positive và negative
        p_w_pos = (freq_pos + 1) / (N_pos + V)
        p_w_neg = (freq_neg + 1) / (N_neg + V)

        # Lưu vào p_likelihood dictionary
        p_likelihood[word] = {0:p_w_neg, 1:p_w_pos}
    # END CODE HERE

    return p_likelihood

In [20]:
def train_naive_bayes(train_x, train_y):
    ''' Huấn luyện thuật toán Naive Bayes
    Args:
        train_x: Danh sách các câu
        train_y: Danh sách các nhãn tương ứng (0 hoặc 1)
    Output:
        p_prior: the prior probability (Xác suấ tiên nghiệm)
        p_likelihood: the maximum likelihood of the probability.
    '''
    # Xây dựng từ điển tần suất xuất hiện của từ và nhãn tương ứng
    freqs = count_freq_words(train_x, train_y)

    # Tính xác suất tiên nghiệm
    p_prior = compute_prior_prob(train_y)

    # Tính xác suất likelihood
    p_likelihood = compute_likelihood(freqs)

    return p_prior, p_likelihood

In [29]:
def naive_bayes_predict(sentence, p_prior, p_likelihood):
    '''
    Args:
        sentence: a string
        p_prior: a dictionary of the prior probability
        p_likelihood: a dictionary of words mapping to the probability
    Output:
        p: the probability of sentence with 0: negative, 1: positive

    '''
    # Tiền xử lý dữ liệu
    words = basic_preprocess(sentence)

    # Khởi tạo giá trị xác suất ban đầu là giá trị xác suất tiên nghiệm
    p_neg = p_prior[0]
    p_pos = p_prior[1]

    for word in words:
        # Kiểm tra xem word có tồn tại trong p_likelihood hay không
        if word in p_likelihood:
            ### START CODE HERE
            # nhân xác suất tiên nghiệm với xác suất likelihood của các từ
            p_neg *= p_likelihood[word][0]
            p_pos *= p_likelihood[word][1]
            ### END CODE HERE
    return {'prob': {0: p_neg, 1: p_pos},
            'label': 0 if p_neg > p_pos else 1}

# Training

In [21]:
p_prior, p_likelihood = train_naive_bayes(train_x, train_y)

V: 10840, N_pos: 41915, N_neg: 43321


#Predict

In [26]:
test_x[0]

'Bro:U wan cut hair anot,ur hair long Liao bo\nMe:since ord liao,take it easy lor treat as save $ leave it longer :)\nBro:LOL Sibei xialan'

In [27]:
test_y[0]

1.0

In [30]:
naive_bayes_predict(test_x[0], p_prior, p_likelihood)

{'prob': {0: 1.103764144713296e-80, 1: 2.101225984564381e-79}, 'label': 1}

# Testing accuracy

In [35]:
acc = 0
for sentence, label in zip(test_x, test_y):
  pred = naive_bayes_predict(sentence, p_prior, p_likelihood)['label']

  if pred == label:
    acc+=1
print("Accuracy", acc/len(test_x))

Accuracy 0.994
